### Imports

In [ ]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import math
import torch
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

### Generation

In [ ]:
"""
Load in fine-tuned ProtGPT2 model and tokenizer. Generate with a higher repetition penality of 1.5 to prevent the model from generating repetitive sequences.
A broader range of sequences will increase the likelihood of stable anti-crisprs.
"""

In [ ]:
# load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/Duke/Freshman Year/Sem 2/BME 590/Shrey Goel/Individual Project 2A/finetuned_protgpt2_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# generate sequences
protgpt2 = pipeline('text-generation', model=model_path)
sequences10 = protgpt2("<|endoftext|>", max_length=100, do_sample=True, top_k=950, repetition_penalty=1.5, num_return_sequences=1000, eos_token_id=0)

In [ ]:
# store only sequences from generation output in a list
gen_seqs10 = []
for item in sequences10:
  gen_seqs10.extend(item.values())

In [ ]:
"""
Perplexity is calculated as a way to assess the model's confidence in its generated sequences and how confused it is in its outputs.
"""

In [ ]:
# function to calculate perplexity of each generated sequence
def calculatePerplexity(sequence, model, tokenizer):
    input_ids = torch.tensor(tokenizer.encode(sequence)).unsqueeze(0)
    input_ids = input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return math.exp(loss)

# calculate pPL for sequences
gen_ppls10 = []
for sequence in gen_seqs10:
  ppl = calculatePerplexity(sequence, model.to(device), tokenizer)
  gen_ppls10.append(ppl)

In [ ]:
# save sequences and associated pPL to df
import pandas as pd

df10 = pd.DataFrame(columns=['Sequence', 'Perplexity'])
df10['Sequence'] = gen_seqs10
df10['Perplexity'] = gen_ppls10

In [ ]:
df10

,Sequence,Perplexity
0,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAITVTPSGEVSH...,15.110159
1,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107
2,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAITVTPSGEVSH...,1.388807
3,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107
4,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.557050
...,...,...
995,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIFATPEGEVSH...,8.729079
996,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107
997,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,5.464747
998,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107


In [ ]:
import pickle
dir = "/content/drive/MyDrive/Duke/Freshman Year/Sem 2/BME 590/Shrey Goel/Individual Project 2A/finetuned_protgpt2_gen_results/"
with open(dir + 'df10.pkl', 'wb') as f:
    pickle.dump(df10, f)